In [15]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sqlalchemy import create_engine
# Magic command to load ipython-sq

In [16]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
import psycopg2

In [17]:
ml_df = pd.read_csv("2.0_ML_Data.csv")
ml_df

,FIPS,State,County Name,Urban Rural Code,Urban Rural Description,Population Estimate 2020,MEDIAN_AGE_TOT,Total Vaccines,Percent Vaccinated,Case Count,Percent of Population Cases,COVID-19 Deaths,Percent_Pop_Death_by_Covid,Pct_Pop_Died_by_Covid_Above_Median
0,6113,CA,Yolo County,2.0,Large fringe metro,219728.0,31.3,132599.0,60.346883,19001.0,8.647510,203.0,0.092387,0.0
1,8041,CO,El Paso County,3.0,Medium metro,728310.0,34.6,390344.0,53.595859,83725.0,11.495792,1234.0,0.169433,0.0
2,17029,IL,Coles County,5.0,Micropolitan,50383.0,37.0,19345.0,38.395888,7265.0,14.419546,140.0,0.277872,1.0
3,13059,GA,Clarke County,4.0,Small metro,127795.0,28.8,29461.0,23.053328,13717.0,10.733597,550.0,0.430377,1.0
4,12086,FL,Miami-Dade County,1.0,Large central metro,2707303.0,40.4,2027846.0,74.902809,491834.0,18.166936,7994.0,0.295275,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,13215,GA,Muscogee County,3.0,Medium metro,196442.0,34.5,62248.0,31.687725,16312.0,8.303723,926.0,0.471386,1.0
795,51540,VA,Charlottesville city,4.0,Small metro,46950.0,32.1,22537.0,48.002130,4472.0,9.525027,204.0,0.434505,1.0
796,39155,OH,Trumbull County,3.0,Medium metro,196800.0,44.6,97110.0,49.344512,18896.0,9.601626,441.0,0.224085,0.0
797,42005,PA,Armstrong County,2.0,Large fringe metro,64162.0,47.0,34269.0,53.410118,6865.0,10.699479,118.0,0.183909,0.0


In [18]:
machine_learning_df = ml_df.drop(["FIPS", "State", "County Name", "Urban Rural Description", "COVID-19 Deaths", "Percent_Pop_Death_by_Covid"], axis=1)
machine_learning_df
# machine_learning_df.to_csv('Model_Machine_Learning_Data.csv', index=False)

,Urban Rural Code,Population Estimate 2020,MEDIAN_AGE_TOT,Total Vaccines,Percent Vaccinated,Case Count,Percent of Population Cases,Pct_Pop_Died_by_Covid_Above_Median
0,2.0,219728.0,31.3,132599.0,60.346883,19001.0,8.647510,0.0
1,3.0,728310.0,34.6,390344.0,53.595859,83725.0,11.495792,0.0
2,5.0,50383.0,37.0,19345.0,38.395888,7265.0,14.419546,1.0
3,4.0,127795.0,28.8,29461.0,23.053328,13717.0,10.733597,1.0
4,1.0,2707303.0,40.4,2027846.0,74.902809,491834.0,18.166936,1.0
...,...,...,...,...,...,...,...,...
794,3.0,196442.0,34.5,62248.0,31.687725,16312.0,8.303723,1.0
795,4.0,46950.0,32.1,22537.0,48.002130,4472.0,9.525027,1.0
796,3.0,196800.0,44.6,97110.0,49.344512,18896.0,9.601626,0.0
797,2.0,64162.0,47.0,34269.0,53.410118,6865.0,10.699479,0.0


In [19]:
X = machine_learning_df.copy()
X = X.drop("Pct_Pop_Died_by_Covid_Above_Median", axis=1)


# # # # Create our target
y = machine_learning_df["Pct_Pop_Died_by_Covid_Above_Median"]

In [20]:
X.describe()

,Urban Rural Code,Population Estimate 2020,MEDIAN_AGE_TOT,Total Vaccines,Percent Vaccinated,Case Count,Percent of Population Cases
count,799.000000,7.990000e+02,799.000000,7.990000e+02,799.000000,7.990000e+02,799.000000
mean,3.342929,3.105749e+05,39.816020,1.730009e+05,49.484180,3.444012e+04,11.345784
std,1.321709,5.665322e+05,4.619412,3.435429e+05,11.877537,7.439737e+04,3.769642
min,1.000000,2.045200e+04,25.200000,2.254000e+03,5.157531,2.000000e+00,0.002340
25%,2.000000,7.594600e+04,37.000000,3.310850e+04,42.428624,8.464000e+03,9.214834
50%,3.000000,1.519460e+05,39.600000,7.262400e+04,49.768712,1.534600e+04,11.431696
75%,4.000000,3.262150e+05,42.100000,1.810910e+05,57.316540,3.270450e+04,14.012820
max,6.000000,9.943046e+06,68.100000,6.131044e+06,101.577149,1.429615e+06,20.554329


In [21]:
# Check the balance of our target values
Counter(y)

Counter({0.0: 398, 1.0: 401})

In [22]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [23]:
#Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=250, random_state=1)
eec_model = eec.fit(X_train, y_train)

In [24]:
# Calculated the balanced accuracy score
predictions = eec_model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.6

In [25]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[63, 37],
       [43, 57]], dtype=int64)

In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.59      0.63      0.57      0.61      0.60      0.36       100
        1.0       0.61      0.57      0.63      0.59      0.60      0.36       100

avg / total       0.60      0.60      0.60      0.60      0.60      0.36       200

